In [1]:
import glob
import skimage
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde
plt.style.use('default')

In [2]:
class Resize:
    
    def resize(self, output_height, output_width):
        if hasattr(self, 'data'):
            self.data = (
                skimage.transform.resize(self.data, 
                                         output_shape = (output_height, output_width), 
                                         anti_aliasing = True)
            )
            self.height, self.width = self._update_hw()
        elif hasattr(self, 'bbox_image'):
            self.bbox_image = (
                skimage.transform.resize(self.bbox_image, 
                                         output_shape = (output_height, output_width), 
                                         anti_aliasing = True)
            )
        else:
            raise AttributeError("No applicable attribute for this method.")   
        return self
    
    def _update_hw(self):
        height, width = self.data.shape[0], self.data.shape[1] 
        return height, width

class Pic(Resize):
    
    C_MAP = None
    
    def __init__(self, local_file = None, data = None, yolo_coords = None):
        if local_file is not None:
            self.data = skimage.io.imread(local_file)
        elif data is not None:
            self.data = data
        else:
            raise TypeError("Missing at least one required argument!")
        self._original_copy = self.data
        self.height, self.width = self._update_hw()
        self.footprint = skimage.morphology.square(2)
        if yolo_coords is not None:
            self.associated_yolo_coords = yolo_coords
            self.associated_yolo_coords.associate_pic(self).coords_to_pixels().bbox_image()
        
    @classmethod
    def get_cmap(cls):
        return cls.C_MAP 
    
    def reset(self):
        self.data = self._original_copy
        return self
        
    def show_pic(self, contours = None, blobs = None):
        dpi = 200
        plt.figure(dpi = dpi)
        plt.grid(False)
        plt.imshow(self.data, cmap = self.get_cmap())
        if contours is not None:
            for contour in contours:
                plt.plot(contour[:, 1], 
                         contour[:, 0], 
                         linewidth = 1)
        ax = plt.gca()
        if blobs is not None:
            for blob in blobs:
                y, x, area = blob
                ax.add_patch(plt.Circle((x, y), 
                                        area, 
                                        color = 'r', 
                                        fill = False, 
                                        linewidth = 1))
        if hasattr(self, 'associated_yolo_coords'):
            for k, v in self.associated_yolo_coords.pixel_coords_dict.items():
                for v_2 in v:
                    if k == 0:
                        edgecolor = 'red'
                    else:
                        edgecolor = 'blue'
                    rect = Rectangle((v_2[0], v_2[1]), v_2[2], v_2[3],
                                     linewidth = 2,
                                     edgecolor = edgecolor,
                                     facecolor = 'none')
                    ax.add_patch(rect)
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)
        plt.show()
        
    def show_hist(self):
        dpi = 200
        plt.figure(dpi = dpi)
        plt.hist(self.data.ravel(), 
                 bins = 256, 
                 histtype = 'stepfilled', 
                 color = 'darkblue',
                 edgecolor = 'black',
                 density = True)
        plt.title('Pixel Intensity Histogram')
        plt.xlim(0,1)
        plt.show()
        
    def equalize_contrast(self):
        self.data = skimage.exposure.equalize_hist(self.data)
        return self
        
    def clahe(self, clip_limit = 0.03):
        self.data = skimage.exposure.equalize_adapthist(self.data, clip_limit = clip_limit)
        return self
    
    def hog(self):
        if self.get_cmap() == 'gray':
            channel_axis = None
        else:
            channel_axis = -1
        _, hog_image = skimage.feature.hog(self.data, 
                                           orientations = 8, 
                                           pixels_per_cell = (16, 16),
                                           cells_per_block = (1, 1), 
                                           visualize = True,
                                           channel_axis = channel_axis)
        self.data = skimage.exposure.rescale_intensity(hog_image, 
                                                       in_range = 'image', 
                                                       out_range = (0, 50))
        return self.show_pic()

    def enhance_contrast(self):
        self = self.turn_to_ubyte()
        self.data = skimage.filters.rank.enhance_contrast(self.data, self.footprint)
        return self
    
    def turn_to_ubyte(self):
        self.data = skimage.util.img_as_ubyte(self.data)
        return self

In [3]:
class YoloCoords(Resize):
    
    def __init__(self, local_file):
        with open(coords, 'r', encoding = 'utf-8') as file:
            raw_data = file.read()
            file.close()
        split_raw_data = str.split(raw_data, sep = '\n')
        self.class_coords_dict = defaultdict(list)
        for i in split_raw_data:
            split_string = str.split(i, sep = ' ')
            if split_string != ['']:
                string_into_floats = [float(j) for j in split_string]
                self.class_coords_dict[string_into_floats[0]].append(string_into_floats[1:5])
                
    def associate_pic(self, pic):
        self.associated_pic = pic
        return self
                
    def coords_to_pixels(self):
        self.pixel_coords_dict = defaultdict(list)
        for k,v in self.class_coords_dict.items():
            for v_2 in v:
                center_x = v_2[0] * self.associated_pic.width
                center_y = v_2[1] * self.associated_pic.height
                box_width = v_2[2] * self.associated_pic.width
                box_height = v_2[3] * self.associated_pic.height
                origin_x = center_x - box_width/2
                origin_y = center_y - box_height/2
                ls_of_values = [origin_x, origin_y, box_width, box_height]
                ls_of_values = [round(i) for i in ls_of_values]
                self.pixel_coords_dict[k].append(ls_of_values)
        return self
    
    def bbox_image(self):
        self.bbox_image = np.zeros((self.associated_pic.height, 
                                    self.associated_pic.width), 
                                   dtype = int)
        for k,v in yolo_coords.pixel_coords_dict.items():
            for v_2 in v:
                origin_x, origin_y, width, height = v_2[0], v_2[1], v_2[2], v_2[3]
                other_side = origin_x + width
                top_height = origin_y + height
                self.bbox_image[origin_y, origin_x:other_side] = 1
                self.bbox_image[origin_y + height, origin_x:other_side] = 1
                self.bbox_image[origin_y:top_height, origin_x] = 1
                self.bbox_image[origin_y:top_height, origin_x + width] = 1
        return self

In [4]:
class ColorPic(Pic):
    
    def __init__(self, local_file = None, data = None, yolo_coords = None):
        super().__init__(local_file, data, list_yolo_coords)
        
    def show_rgb_hist(self):
        dpi = 200
        plt.figure(dpi = dpi)
        plt.hist(self.data[:,:,0].ravel(), bins = 256, color = 'red', alpha = 0.5, density = True)
        plt.hist(self.data[:,:,1].ravel(), bins = 256, color = 'green', alpha = 0.5, density = True)
        plt.hist(self.data[:,:,2].ravel(), bins = 256, color = 'blue', alpha = 0.5, density = True)
        plt.xlim(0, 256)
        plt.title('Density of RGB Values in Picture')
        plt.show()
        
    def slic(self, n_segments = 500):
        segments = skimage.segmentation.slic(self.data, 
                                             n_segments = n_segments, 
                                             compactness = 1,
                                             sigma = 1)
        self.data = skimage.color.label2rgb(segments, self.data, kind = 'avg')
        return self
    
    def reset_to_gray(self):
        return GrayPic(local_file = None, data = self.reset().data)

In [5]:
class GrayPic(Pic):
    
    C_MAP = 'gray'
    
    def __init__(self, local_file = None, data = None, yolo_coords = None):
        super().__init__(local_file, data, list_yolo_coords)
        if len(self.data.shape) == 3:
            self._original_color_copy = self.data
            self.data = skimage.color.rgb2gray(self.data)
            self._original_copy = self.data
        else:
            self._original_color_copy = None
            self.data = self.data
            self._original_copy = self.data
        self.global_thresh = skimage.filters.threshold_otsu(self.data)
        
    def thresh(self):
        self.data = self.data > self.global_thresh
        return self
        
    def inverse_thresh(self):
        self.data = self.data <= self.global_thresh
        return self
    
    def edges(self, sigma = 0.5):
        self.data = skimage.feature.canny(self.data, sigma = sigma)
        return self
    
    def erode(self):
        self.data = skimage.morphology.binary_erosion(self.data)
        return self
    
    def dilate(self):
        self.data = skimage.morphology.binary_dilation(self.data)
        return self
    
    def close(self):
        self.data = skimage.morphology.binary_closing(self.data, self.footprint)
        return self
    
    def show_contours(self):
        contours = skimage.measure.find_contours(self.thresh().data)
        return self.show_pic(contours = contours)
      
    def show_blobs(self):
        blobs = skimage.feature.blob_doh(self.thresh().close().data)
        return self.show_pic(blobs = blobs)
    
# Fix with gradients.
    def watershed(self):
        edges = skimage.filters.sobel(self.data)
        markers = np.zeros_like(self.data)
        foreground, background = 1, 2
        markers[self.data <= np.quantile(self.data, 0.25)] = background
        markers[self.data > np.quantile(self.data, 0.25)] = foreground

        ws = skimage.segmentation.watershed(edges, markers)
        segments = skimage.measure.label(ws == foreground)
        segments_colors = skimage.color.label2rgb(segments, image = self.data, bg_label = 0)
        
        plt.grid(False)
        plt.imshow(segments_colors)
        ax = plt.gca()
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)
        plt.show()
        
    def back_to_color_reset(self):
        if self._original_color_copy is not None:
            return ColorPic(local_file = None, data = self._original_color_copy)
        else:
            raise ValueError('Originally in grayscale.')
            return self.reset()

In [6]:
class PicCollectionDf(pd.DataFrame):
    
    def __init__(self, rows, columns):
        super().__init__(data = rows, columns = columns)

In [7]:
class PicCollection:
    
    def __init__(self, ls_of_pics):
        self.collection = []
        for i in ls_of_pics:
            if isinstance(i, Pic):
                self.collection.append(i)
            else:
                raise ValueError("PicCollection must contain a Picture!")
        self.sizes = {i: i.height * i.width for i in self.collection}
        self.min_size_object = sorted(self.sizes.items(), 
                                      key = lambda x: x[1])[0]
        self.max_size_object = sorted(self.sizes.items(), 
                                      key = lambda x: x[1],
                                      reverse = True)[0]
        
    def resize_collection(self, min_or_max = 'min'):
        if min_or_max == 'max':
            output_height = output_width = np.sqrt(self.max_size_object[1])
        else:
            output_height = output_width = np.sqrt(self.min_size_object[1]) * 1.33
        self.resized_collection = (
            [i.resize(output_height = output_height,
                      output_width = output_width).turn_to_ubyte() for i in self.collection]
        )
        return self
    
    def collection_to_df(self):
        self.resize_collection()
        cols_ls = []
        for i in range(self.resized_collection[0].height):
            for j in range(self.resized_collection[0].width):
                cols_ls.append(f"Pixel_{i}_{j}")
        rows_of_df = [i.data.ravel() for i in self.resized_collection]
        return PicCollectionDf(rows = rows_of_df, columns = cols_ls)

In [8]:
# pic = glob.glob('data/original_pictures/*.jpg')[0]
# coords = "C:/Users/charr/Desktop/0185e0011277f3942c7a96be719d5d9150907cb4.xml.txt"
# yolo_coords = YoloCoords(coords)
# pic = Pic(local_file = pic, yolo_coords = yolo_coords)

In [9]:
# collection = PicCollection([ColorPic(i) for i in glob.glob('data/original_pictures/*.jpg')])

In [10]:
# various_sizes = list(collection.sizes.values())

# density = gaussian_kde(various_sizes)
# min_size = np.min(various_sizes)
# max_size = np.max(various_sizes)
# density.covariance_factor = lambda : .25
# density._compute_covariance()
# x_values = np.linspace(min_size, max_size, num = 100)
# plt.plot(x_values, density(x_values), c = 'black')
# plt.xlim(100000,400000)
# plt.ylim(0, 10e-6)
# plt.title('Density of Various Picture Sizes')
# plt.xlabel('Total Pixel Area')
# plt.show();